## Smoothing path

<img src="L5/1.png" alt="Drawing" style="width: 400px;"/>
* Smooth the path by minimizing the quantities $||x_i - y_i||^2$ and $||y_i - y_{i+1}||^2$ for all $i$, where $x_i$ and $y_i$ are the original and smooth points.
* If only $min(||x_i - y_i||^2)$, back to the orginal path.
* If only $min(||y_i - y_{i+1}||^2)$, collapse to one single point.
* What if $||x_i - y_i||^2 + \alpha||y_i - y_{i+1}||^2$, results in a smooth path. In practice, we optimize $c_1||x_i - y_i||^2 + c_2||y_i - y_{i+1}||^2$

* ### Gradient decent
For i between 1 and n-1, $y_i = y_i + \alpha(x_i-y_i) + \beta(y_{i-1}-2y_i+y_{i+1}$).
* Note: $x_0 == y_0$ and $x_n == y_n$


In [14]:
from math import *

path = [[0, 0],
        [0, 1],
        [0, 2],
        [1, 2],
        [2, 2],
        [3, 2],
        [4, 2],
        [4, 3],
        [4, 4]]

def smooth(path, weight_data = 0.5, weight_smooth = 0.1, tolerance = 1e-6):
    # deep copy
    newpath = [[0 for col in range(len(path[0]))]
               for row in range(len(path))]
    for i in range(len(path)):
        for j in range(len(path[0])):
            newpath[i][j] = path[i][j]
            
    
    change = tolerance
    while (change >= tolerance):
        change = 0
        for i in range(1,len(path)-1):
            for j in range(len(path[0])):
                  d1 = weight_data * (path[i][j]-newpath[i][j])
                  d2 = weight_smooth * (newpath[i-1][j] +
                                        newpath[i+1][j] -
                                        2 * newpath[i][j])
                  change += abs(d1 + d2)
                  newpath[i][j] += d1 + d2
    return newpath


# set weight_data=0, gives a strange line.
# because the first and last points are fixed.
smooth(path, weight_data=0)


[[0, 0],
 [0.5000022870105129, 0.49999771298948714],
 [1.0000041902374124, 0.9999958097625875],
 [1.5000054286782385, 1.4999945713217615],
 [2.0000058264481124, 1.9999941735518876],
 [2.5000053375795015, 2.499994662420499],
 [3.0000040507845105, 2.99999594921549],
 [3.5000021737967075, 3.4999978262032925],
 [4, 4]]

## PID control

* P (proportional)

<img src="L5/2.png" alt="Drawing" style="width: 400px;"/>

<img src="L5/3.png" alt="Drawing" style="width: 400px;"/>

The steering angle $\alpha = - \tau CTE_t$, where $CTE_t$ is the cross track error.

**Marginally stable**: although the steering is straight, the car is already pointing into the trajectory when it reaches the trajectory. This happens no matter how small   is, and because of that the car is forced to overshoot. This results in the **Oscillations**.

In [28]:
from robot_pid import robot

def run(param=0.2):
    myrobot = robot()
    myrobot.set(0., 1., 0.)
    speed = 1.0
    N = 1000
    for i in range(N):
        cte = myrobot.y
        steer = -param * cte
        myrobot = myrobot.move(steer, speed)
        print myrobot, steer

run(0.3)

#run(0.1)
# The oscillation is much slower and 
# the cross-track error compensation is much less.

[x=0.99996 y=0.99227 orient=6.26772] -0.3
[x=1.99968 y=0.96913 orient=6.25238] -0.297680024378
[x=2.99894 y=0.93085 orient=6.23742] -0.290739697204
[x=3.99753 y=0.87794 orient=6.22308] -0.279256486166
[x=4.99529 y=0.81115 orient=6.20960] -0.263383153666
[x=5.99210 y=0.73144 orient=6.19718] -0.243344375771
[x=6.98791 y=0.63999 orient=6.18603] -0.219431725295
[x=7.98270 y=0.53816 orient=6.17631] -0.191997398339
[x=8.97655 y=0.42744 orient=6.16817] -0.161447107227
[x=9.96957 y=0.30948 orient=6.16172] -0.128232547131
[x=10.96192 y=0.18601 orient=6.15707] -0.0928437794261
[x=11.95380 y=0.05884 orient=6.15428] -0.0558017788004
[x=12.94550 y=-0.06972 orient=6.15339] -0.0176512818527
[x=13.93716 y=-0.19863 orient=6.15444] 0.0209147370566
[x=14.92907 y=-0.32554 orient=6.15742] 0.0595877696301
[x=15.92148 y=-0.44854 orient=6.16232] 0.0976614327046
[x=16.91458 y=-0.56575 orient=6.16909] 0.134562103709
[x=17.90855 y=-0.67534 orient=6.17766] 0.169725266537
[x=18.90351 y=-0.77557 orient=6.18793] 0.2

* D (differential)

To overcome the steering drift, $\alpha = - \tau_p CTE_t - \tau_d \frac{d}{dt}CTE_t$, where $\frac{d}{dt}CTE_t = \frac{CTE_t - CTE_{t-\Delta}}{\Delta t}$

In [32]:
from robot_pid import robot

def run(param1=0.2, param2=3.0):
    myrobot = robot()
    myrobot.set(0., 1., 0.)
    speed = 1.0
    N = 1000
    cte = myrobot.y
    
    for i in range(N):
        diff_cte = myrobot.y - cte
        cte = myrobot.y
        steer = - param1 * cte - param2 * diff_cte
        myrobot = myrobot.move(steer, speed)
        print myrobot, steer

run(0.3, 3)

[x=0.99996 y=0.99227 orient=6.26772] -0.3
[x=1.99970 y=0.96976 orient=6.25364] -0.274480268154
[x=2.99907 y=0.93454 orient=6.24228] -0.223415450442
[x=3.99804 y=0.88924 orient=6.23345] -0.174706236533
[x=4.99664 y=0.83624 orient=6.22687] -0.130861152071
[x=5.99492 y=0.77766 orient=6.22227] -0.0918808742386
[x=6.99297 y=0.71534 orient=6.21939] -0.0575506448292
[x=7.99089 y=0.65089 orient=6.21800] -0.0276436576228
[x=8.98877 y=0.58576 orient=6.21791] -0.00192753835752
[x=9.98664 y=0.52052 orient=6.21889] 0.0196829252408
[x=10.98464 y=0.45726 orient=6.22087] 0.039541296159
[x=11.98278 y=0.39630 orient=6.22350] 0.0526126116902
[x=12.98109 y=0.33825 orient=6.22671] 0.0639979506253
[x=13.97960 y=0.28362 orient=6.23035] 0.0726721353404
[x=14.97830 y=0.23277 orient=6.23429] 0.07881167032
[x=15.97721 y=0.18597 orient=6.23844] 0.0826972814011
[x=16.97630 y=0.14336 orient=6.24268] 0.0846125753969
[x=17.97556 y=0.10499 orient=6.24693] 0.084831487158
[x=18.97498 y=0.07084 orient=6.25112] 0.08361424

* I (integral)

An everyday example of systematic bias is when the wheels of a car are out of alignment. 

To overcome the systematic bias from the PD, $\alpha = - \tau_p CTE_t - \tau_i \sum CTE_t - \tau_d \frac{d}{dt}CTE_t$, called **PID**.

In [42]:
from robot_pid import robot

def run(param1, param2, param3):
    myrobot = robot()
    myrobot.set(0., 1., 0.)
    speed = 1.0
    N = 1000
    myrobot.set_steering_drift(10.0 / 180.0 * pi)
    cte = myrobot.y
    int_cte = 0
    
    for i in range(N):
        int_cte += cte
        diff_cte = myrobot.y - cte
        cte = myrobot.y
        steer = - param1 * cte - param2 * diff_cte  - param3 * int_cte
        myrobot = myrobot.move(steer, speed)
        print myrobot, steer

run(0.2, 3, 0.01)

[x=1.00000 y=0.99911 orient=6.28141] -0.21
[x=2.00000 y=0.99627 orient=6.27928] -0.217161444441
[x=2.99998 y=0.99121 orient=6.27697] -0.220724272378
[x=3.99995 y=0.98378 orient=6.27454] -0.223008678836
[x=4.99991 y=0.97389 orient=6.27205] -0.224329412983
[x=5.99983 y=0.96150 orient=6.26953] -0.224813229289
[x=6.99972 y=0.94659 orient=6.26703] -0.224561897803
[x=7.99957 y=0.92921 orient=6.26457] -0.223667467424
[x=8.99937 y=0.90940 orient=6.26219] -0.222214261086
[x=9.99912 y=0.88726 orient=6.25990] -0.220279622686
[x=10.99883 y=0.86289 orient=6.25773] -0.217934497104
[x=11.99848 y=0.83641 orient=6.25569] -0.215243945647
[x=12.99807 y=0.80798 orient=6.25380] -0.212267610631
[x=13.99761 y=0.77773 orient=6.25207] -0.209060134264
[x=14.99711 y=0.74585 orient=6.25052] -0.205671535833
[x=15.99655 y=0.71250 orient=6.24914] -0.202147550898
[x=16.99595 y=0.67785 orient=6.24794] -0.198529936006
[x=17.99531 y=0.64210 orient=6.24692] -0.194856742363
[x=18.99465 y=0.60584 orient=6.24609] -0.1911625

## Parameter optimization (Controls gains)

* Twiddle algorithm

In [53]:
# The last a few lines of codes may not be right.

from robot_pid import robot

def run(param, printflag=False):
    myrobot = robot()
    myrobot.set(0., 1., 0.)
    speed = 1.0
    N = 1000
    myrobot.set_steering_drift(10.0 / 180.0 * pi)
    cte = myrobot.y
    int_cte = 0
    err = 0
    
    for i in range(N):
        int_cte += cte
        diff_cte = myrobot.y - cte
        cte = myrobot.y
        steer = - param[0] * cte - param[1] * diff_cte  - param[2] * int_cte
        myrobot = myrobot.move(steer, speed)
        
        if i >= N:
            err += (cte ** 2)
        if printflag:
            print myrobot, steer
    return err / float(N)
            

def twiddle(tol = 0.2): #Make this tolerance bigger if you’re timing out!
    n_params = 3
    dparams = [1.0 for row in range(n_params)] 
    params = [0.0 for row in range(n_params)] 
    best_error = run(params)
    n=0
    while sum(dparams) > tol:
        for i in range(len(params)):
            params[i] += dparams[i]
            err = run(params)
            if err < best_error:
                best_error = err
                dparams[i] *= 1.1
            else:
                params[i] -= 2.0 * dparams[i]
                err = run(params)
                if err < best_error:
                    best_error = err
                    dparams[i] *= 1.1
                else:
                    params[i] += dparams[i]
                    dparams[i] *= 0.9
    n += 1
    print 'Twiddle #', n, params, ' -> ', best_error
    return run(params)

twiddle(tol=0.1)

Twiddle # 1 [0.0, 0.0, 0.0]  ->  0.0


0.0